# Differences between ePACT2 and DataLab High-dose Inhaled Corticosteroid indicators

Both NHSBSA ePACT2 and the DataLab have high-dose inhaled corticosteroid markers.  We have been contacted by people who have queried why the results appear to be very different (e.g. https://twitter.com/TPMPharm/status/1187022788630142976)

The measure from the DataLab is based on table 12 of the latest BTS/SIGN guidance on the treatment of asthma (https://www.brit-thoracic.org.uk/document-library/guidelines/asthma/btssign-guideline-for-the-management-of-asthma-2019/).

I have obtained the list of BNF codes from the DataLab GitHub for the measure and the ePACT2 code list from their measures page (which appear to require a login).  Please note, the BNF codes look different - at the DataLab we tend to use wildcards based on the chemical substance and generic presentation code, to ensure we pick up all brands and generic of the same presentation, whereas the published NHSBSA code is specific to each generic/brand.

Please note, this will only show items which have been prescribed in the latest month of prescribing data (August 2019).

In [151]:
#import libraries required for analysis
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
from ebmdatalab import bq
from ebmdatalab import charts
from ebmdatalab import maps
from IPython.display import display

In [74]:
#get data for latest month for datalab high dose ICS
sql = """
SELECT
  bnf_name,
  bnf_code,
  SUM(items) AS items
FROM
  `hscic.normalised_prescribing_standard_latest_month`
WHERE
  bnf_code LIKE '0302000C0%AC' OR --Beclomet Diprop_Inha 250mcg (200D) (brands and generic)
  bnf_code LIKE '0302000C0%AU' OR --Beclomet Diprop_Inha B/A 250mcg (200 D) (brands and generic)
  bnf_code LIKE '0302000C0%BK' OR --Beclomet Diprop_Pdr For Inh 250mcg(100 D (brands and generic)
    bnf_code LIKE '0302000C0%BW' OR --Beclomet Diprop_Inha 250mcg (200 D) CFF (brands and generic)
    bnf_code LIKE '0302000C0%BZ' OR --Beclomet/Formoterol_Inh 200/6 (120D) CFF (brands and generic)
    bnf_code LIKE '0302000K0%AH' OR --Budesonide_Pdr For Inh 400mcg (50 D) (brands and generic)
    bnf_code LIKE '0302000K0%AU' OR --Budesonide/Formoterol_InhaB/A 400/12(60D (brands and generic)
      bnf_code LIKE '0302000N0%AF' OR --Fluticasone Prop_Pdr Inh 250mcg Disk Ref (brands and generic)
      bnf_code LIKE '0302000N0%AP' OR --Fluticasone Prop_Pdr Inh 500mcg Disk+Dev (brands and generic)
      bnf_code LIKE '0302000N0%AU' OR --Fluticasone Prop_Pdr For Inh 500mcg(60D) (brands and generic)
      bnf_code LIKE '0302000N0%AZ' OR --Fluticasone/Salmeterol_Inh 500/50mcg 60D (brands and generic)
      bnf_code LIKE '0302000N0%BC' OR --Fluticasone Prop_Inha 250mcg (120 D) CFF (brands and generic)
      bnf_code LIKE '0302000N0%BG' OR --Fluticasone/Salmeterol_Inh 250/25mcg120D (brands and generic)
      bnf_code LIKE '0302000N0%BK' OR --Fluticasone/Formoterol_Inh 250/10mcg120D (brands and generic)
      bnf_code LIKE '0302000U0%AB' OR --Ciclesonide_Inh 160mcg (120 D) CFF (brands and generic)
      bnf_code LIKE '0302000U0%AC' OR --Ciclesonide_Inh 160mcg (60 D) CFF (brands and generic)
      bnf_code LIKE '0302000V0%AA'    --Fluticasone/Vilanterol_Inha 184/22mcg30D (brands and generic)
    GROUP BY
      bnf_name,
      bnf_code
    ORDER BY
      items DESC
"""
dl_ics_high_dose_df = bq.cached_read(sql, csv_path='dl_ics_high_dose_df.csv', use_cache=False)

In [144]:
#get data for latest month for datalab all ICS
sql = """
SELECT
  bnf_name,
  bnf_code,
  SUM(items) AS items
FROM
  `hscic.normalised_prescribing_standard_latest_month`
WHERE
    bnf_code LIKE '0302000C0%' OR --Beclometasone Dipropionate
    bnf_code LIKE '0302000K0%' OR --Budesonide
    bnf_code LIKE '0302000N0%' OR --Fluticasone Propionate (Inh)
    bnf_code LIKE '0302000R0%' OR --Mometasone Furoate
    bnf_code LIKE '0302000U0%' OR --Ciclesonide
    bnf_code LIKE '0302000V0%' --Fluticasone Furoate (Inh)
    GROUP BY
      bnf_name,
      bnf_code
    ORDER BY
      items DESC
"""
dl_ics_all_df = bq.cached_read(sql, csv_path='dl_ics_all_df.csv')
dl_ics_all_df2 = dl_ics_all_df  

In [76]:
#get data for latest month for ePACT2 high dose ICS
sql = """
SELECT
  bnf_name,
  bnf_code,
  SUM(items) AS items
FROM
  `hscic.normalised_prescribing_standard_latest_month`
WHERE
bnf_code ='0302000N0BHAAAZ' OR --Aerivio Spiromax_Inh 500/50mcg (60D) (0302000N0BHAAAZ)
bnf_code ='0302000C0BEABAU' OR --Aerobec Fte 250_Autohaler (200 D) (0302000C0BEABAU)
bnf_code ='0302000N0BGACBG' OR --AirFluSal_Inh 250mcg/25mcg (120D) (0302000N0BGACBG)
bnf_code ='0302000N0BJABBG' OR --Aloflute_Inh 250mcg/25mcg (120D) (0302000N0BJABBG)
bnf_code ='0302000C0BIAFBK' OR --Asmabec Clickhaler_D/P Inh 250mcg(100 D) (0302000C0BIAFBK)
bnf_code ='0302000C0BIACBC' OR --Asmabec Spacehaler_Inha 250mcg (200 D) (0302000C0BIACBC)
bnf_code ='0302000R0BBACAC' OR --Asmanex Twisthaler_D/P Inh 400mcg (30 D) (0302000R0BBACAC)
bnf_code ='0302000R0BBADAD' OR --Asmanex Twisthaler_D/P Inh 400mcg (60 D) (0302000R0BBADAD)
bnf_code ='0302000C0BFAFAU' OR --Beclazone 250 E-Breathe_Inha 250mcg(200D (0302000C0BFAFAU)
bnf_code ='0302000C0BFACAC' OR --Beclazone 250_Inha 250mcg (200 D) (0302000C0BFACAC)
bnf_code ='0302000C0BBAIAU' OR --Becloforte E-Breathe_Inha 250mcg (200 D) (0302000C0BBAIAU)
bnf_code ='0302000C0BBAGAY' OR --Becloforte Integra_Inha 250mcg + Spacer (0302000C0BBAGAY)
bnf_code ='0302000C0BBAHAZ' OR --Becloforte Integra_Inha 250mcg Ref (0302000C0BBAHAZ)
bnf_code ='0302000C0BBAFA0' OR --Becloforte-VM_Inha (80 D) + Volumatic (0302000C0BBAFA0)
bnf_code ='0302000C0BBACAJ' OR --Becloforte_Disk 400mcg & Diskhaler (0302000C0BBACAJ)
bnf_code ='0302000C0BBADAQ' OR --Becloforte_Disk 400mcg Ref (0302000C0BBADAQ)
bnf_code ='0302000C0BBAJBT' OR --Becloforte_Inha 250mcg (180 D) (Import) (0302000C0BBAJBT)
bnf_code ='0302000C0BBAAAC' OR --Becloforte_Inha 250mcg (200 D) (0302000C0BBAAAC)
bnf_code ='0302000C0BBAEAX' OR --Becloforte_Inha 250mcg (80 D) (0302000C0BBAEAX)
bnf_code ='0302000C0AABTBT' OR --Beclomet Diprop_Inha 250mcg (180 D) (0302000C0AABTBT)
bnf_code ='0302000C0AABWBW' OR --Beclomet Diprop_Inha 250mcg (200 D) CFF (0302000C0AABWBW)
bnf_code ='0302000C0AAAZAZ' OR --Beclomet Diprop_Inha 250mcg (200 D) Ref (0302000C0AAAZAZ)
bnf_code ='0302000C0AAAYAY' OR --Beclomet Diprop_Inha 250mcg (200 D)+Spac (0302000C0AAAYAY)
bnf_code ='0302000C0AAACAC' OR --Beclomet Diprop_Inha 250mcg (200D) (0302000C0AAACAC)
bnf_code ='0302000C0AAAXAX' OR --Beclomet Diprop_Inha 250mcg (80 D) (0302000C0AAAXAX)
bnf_code ='0302000C0AABCBC' OR --Beclomet Diprop_Inha 250mcg 200 D+V/Spac (0302000C0AABCBC)
bnf_code ='0302000C0AABPBP' OR --Beclomet Diprop_Inha 400mcg (100 D) (0302000C0AABPBP)
bnf_code ='0302000C0AAAUAU' OR --Beclomet Diprop_Inha B/A 250mcg (200 D) (0302000C0AAAUAU)
bnf_code ='0302000C0AABSBS' OR --Beclomet Diprop_Inha B/A 400mcg (100 D) (0302000C0AABSBS)
bnf_code ='0302000C0AABKBK' OR --Beclomet Diprop_Pdr For Inh 250mcg(100 D (0302000C0AABKBK)
bnf_code ='0302000C0AAAIAI' OR --Beclomet Diprop_Pdr For Inh Cap 400mcg (0302000C0AAAIAI)
bnf_code ='0302000C0AAAQAQ' OR --Beclomet Diprop_Pdr Inh 400mcg Disk Ref (0302000C0AAAQAQ)
bnf_code ='0302000C0AAAJAJ' OR --Beclomet Diprop_Pdr Inh 400mcg Disk+Dev (0302000C0AAAJAJ)
bnf_code ='0302000C0BMACAI' OR --Beclometasone 400 Cyclocaps_Cap 400mcg (0302000C0BMACAI)
bnf_code ='0302000C0BHAEAJ' OR --Becodisks_Disk 400mcg & Diskhaler (0302000C0BHAEAJ)
bnf_code ='0302000C0BHAFAQ' OR --Becodisks_Disk 400mcg Ref (0302000C0BHAFAQ)
bnf_code ='0302000C0BCANAJ' OR --Becotide_Becodisk 400mcg & Diskhaler (0302000C0BCANAJ)
bnf_code ='0302000C0BCAPAQ' OR --Becotide_Becodisk 400mcg Ref (0302000C0BCAPAQ)
bnf_code ='0302000C0BCAGAI' OR --Becotide_R/Cap 400mcg (0302000C0BCAGAI)
bnf_code ='0302000K0BEABAS' OR --Budesonide 400 Cyclocaps_Cap 400mcg (0302000K0BEABAS)
bnf_code ='0302000K0AAAUAU' OR --Budesonide/Formoterol_InhaB/A 400/12(60D (0302000K0AAAUAU)
bnf_code ='0302000K0AAAYAY' OR --Budesonide_Pdr For Inh 400mcg (100 D) (0302000K0AAAYAY)
bnf_code ='0302000K0AAASAS' OR --Budesonide_Pdr For Inh Cap 400mcg (0302000K0AAASAS)
bnf_code ='0302000C0BPADBW' OR --Clenil Modulite_Inha 250mcg (200D) (0302000C0BPADBW)
bnf_code ='0302000K0BHABAU' OR --DuoResp Spiromax_Inh 320mcg/9mcg (60 D) (0302000K0BHABAU)
bnf_code ='0302000K0BGACAY' OR --Easyhaler_Budesonide 400mcg (100 D) (0302000K0BGACAY)
bnf_code ='0302000C0BGACAC' OR --Filair Fte_Inha 250mcg (200 D) (0302000C0BGACAC)
bnf_code ='0302000N0BBATAT' OR --Flixotide_Accuhaler 250mcg (60 D) (0302000N0BBATAT)
bnf_code ='0302000N0BBAUAU' OR --Flixotide_Accuhaler 500mcg (60 D) (0302000N0BBAUAU)
bnf_code ='0302000N0BBAEAE' OR --Flixotide_Disk 250mcg & Diskhaler (0302000N0BBAEAE)
bnf_code ='0302000N0BBAFAF' OR --Flixotide_Disk 250mcg Ref (0302000N0BBAFAF)
bnf_code ='0302000N0BBAPAP' OR --Flixotide_Disk 500mcg & Diskhaler (0302000N0BBAPAP)
bnf_code ='0302000N0BBAQAQ' OR --Flixotide_Disk 500mcg Ref (0302000N0BBAQAQ)
bnf_code ='0302000N0BBAZBC' OR --Flixotide_Evohaler 250mcg (120 D) (0302000N0BBAZBC)
bnf_code ='0302000N0BBBABD' OR --Flixotide_Evohaler 250mcg (60 D) (0302000N0BBBABD)
bnf_code ='0302000N0BBAMAM' OR --Flixotide_Inha 250mcg (120 D) (0302000N0BBAMAM)
bnf_code ='0302000N0BBANAN' OR --Flixotide_Inha 250mcg (60 D) (0302000N0BBANAN)
bnf_code ='0302000N0AABCBC' OR --Fluticasone Prop_Inha 250mcg (120 D) CFF (0302000N0AABCBC)
bnf_code ='0302000N0AAANAN' OR --Fluticasone Prop_Inha 250mcg (60 D) (0302000N0AAANAN)
bnf_code ='0302000N0AABDBD' OR --Fluticasone Prop_Inha 250mcg (60 D) CFF (0302000N0AABDBD)
bnf_code ='0302000N0AAATAT' OR --Fluticasone Prop_Pdr For Inh 250mcg(60D) (0302000N0AAATAT)
bnf_code ='0302000N0AAAUAU' OR --Fluticasone Prop_Pdr For Inh 500mcg(60D) (0302000N0AAAUAU)
bnf_code ='0302000N0AABIBI' OR --Fluticasone Prop_Pdr Inh 250mcg Disk @gn (0302000N0AABIBI)
bnf_code ='0302000N0AAAFAF' OR --Fluticasone Prop_Pdr Inh 250mcg Disk Ref (0302000N0AAAFAF)
bnf_code ='0302000N0AAAEAE' OR --Fluticasone Prop_Pdr Inh 250mcg Disk+Dev (0302000N0AAAEAE)
bnf_code ='0302000N0AAAQAQ' OR --Fluticasone Prop_Pdr Inh 500mcg Disk Ref (0302000N0AAAQAQ)
bnf_code ='0302000N0AAAPAP' OR --Fluticasone Prop_Pdr Inh 500mcg Disk+Dev (0302000N0AAAPAP)
bnf_code ='0302000N0AABKBK' OR --Fluticasone/Formoterol_Inh 250/10mcg120D (0302000N0AABKBK)
bnf_code ='0302000N0AABGBG' OR --Fluticasone/Salmeterol_Inh 250/25mcg120D (0302000N0AABGBG)
bnf_code ='0302000N0AAAYAY' OR --Fluticasone/Salmeterol_Inh 250/50mcg 60D (0302000N0AAAYAY)
bnf_code ='0302000N0AAAZAZ' OR --Fluticasone/Salmeterol_Inh 500/50mcg 60D (0302000N0AAAZAZ)
bnf_code ='0302000N0BDABBK' OR --Flutiform_Inha 250/10mcg (120 D) (0302000N0BDABBK)
bnf_code ='0302000K0BIAAAU' OR --Fobumix_Easyhaler 320mcg/9mcg (60 D) (0302000K0BIAAAU)
bnf_code ='0302000C0BRAABY' OR --Fostair NEXThaler_Inh 100mcg/6mcg (120D) (0302000C0BRAABY)
bnf_code ='0302000C0BRABCA' OR --Fostair NEXThaler_Inh 200mcg/6mcg (120D) (0302000C0BRABCA)
bnf_code ='0302000C0BQAABX' OR --Fostair_Inh 100mcg/6mcg (120D) CFF (0302000C0BQAABX)
bnf_code ='0302000C0BQABBZ' OR --Fostair_Inh 200mcg/6mcg (120D) CFF (0302000C0BQABBZ)
bnf_code ='0302000R0AAACAC' OR --Mometasone Fur_Pdr For Inh 400mcg (30 D) (0302000R0AAACAC)
bnf_code ='0302000R0AAADAD' OR --Mometasone Fur_Pdr For Inh 400mcg (60 D) (0302000R0AAADAD)
bnf_code ='0302000K0BCAAAH' OR --Pulmaxan_Turbohaler 400mcg (50 D) (0302000K0BCAAAH)
bnf_code ='0302000K0BBAIAH' OR --Pulmicort_Turbohaler 400mcg (50 D) (0302000K0BBAIAH)
bnf_code ='0302000C0BLACBP' OR --Pulvinal Beclomet_Inha 400mcg (100 D) (0302000C0BLACBP)
bnf_code ='0302000V0BBAAAA' OR --Relvar Ellipta_Inha 184mcg/22mcg (30 D) (0302000V0BBAAAA)
bnf_code ='0302000N0BIABBG' OR --Sereflo_Inh 250mcg/25mcg (120D) (0302000N0BIABBG)
bnf_code ='0302000N0BCABAY' OR --Seretide 250_Accuhaler 250mcg/50mcg(60D) (0302000N0BCABAY)
bnf_code ='0302000N0BCAFBG' OR --Seretide 250_Evohaler 250mcg/25mcg(120D) (0302000N0BCAFBG)
bnf_code ='0302000N0BCACAZ' OR --Seretide 500_Accuhaler 500mcg/50mcg(60D) (0302000N0BCACAZ)
bnf_code ='0302000N0BFABBG' OR --Sirdupla_Inh 250mcg/25mcg (120D) (0302000N0BFABBG)
bnf_code ='0302000K0BDACAU' --Symbicort_Turbohaler 400mcg/12mcg (60 D) (0302000K0BDACAU)

    GROUP BY
      bnf_name,
      bnf_code
    ORDER BY
      items DESC
"""
bsa_ics_high_dose_df = bq.cached_read(sql, csv_path='bsa_ics_high_dose_df.csv')

In [143]:
#get data for latest month for ePACT2 all ICS
sql = """
SELECT
  bnf_name,
  bnf_code,
  SUM(items) AS items
FROM
  `hscic.normalised_prescribing_standard_latest_month`
WHERE
bnf_code ='0302000N0BHAAAZ' OR --Aerivio Spiromax_Inh 500/50mcg (60D) (0302000N0BHAAAZ)
bnf_code ='0302000C0BEACAT' OR --Aerobec 100_Autohaler (200 D) (0302000C0BEACAT)
bnf_code ='0302000C0BEAAAS' OR --Aerobec 50_Autohaler (200 D) (0302000C0BEAAAS)
bnf_code ='0302000C0BEABAU' OR --Aerobec Fte 250_Autohaler (200 D) (0302000C0BEABAU)
bnf_code ='0302000N0BGAAAZ' OR --AirFluSal Forspiro_Inh 500/50mcg (60D) (0302000N0BGAAAZ)
bnf_code ='0302000N0BGABBF' OR --AirFluSal_Inh 125mcg/25mcg (120D) (0302000N0BGABBF)
bnf_code ='0302000N0BGACBG' OR --AirFluSal_Inh 250mcg/25mcg (120D) (0302000N0BGACBG)
bnf_code ='0302000N0BJAABF' OR --Aloflute_Inh 125mcg/25mcg (120D) (0302000N0BJAABF)
bnf_code ='0302000N0BJABBG' OR --Aloflute_Inh 250mcg/25mcg (120D) (0302000N0BJABBG)
bnf_code ='0302000U0BBABAB' OR --Alvesco 160_Inh 160mcg (120 D) CFF (0302000U0BBABAB)
bnf_code ='0302000U0BBACAC' OR --Alvesco 160_Inh 160mcg (60 D) CFF (0302000U0BBACAC)
bnf_code ='0302000U0BBAAAA' OR --Alvesco 80_Inh 80mcg (120 D) CFF (0302000U0BBAAAA)
bnf_code ='0302000C0BIAEBJ' OR --Asmabec Clickhaler_D/P Inh 100mcg(200 D) (0302000C0BIAEBJ)
bnf_code ='0302000C0BIAFBK' OR --Asmabec Clickhaler_D/P Inh 250mcg(100 D) (0302000C0BIAFBK)
bnf_code ='0302000C0BIADBI' OR --Asmabec Clickhaler_D/P Inh 50mcg (200 D) (0302000C0BIADBI)
bnf_code ='0302000C0BIABBB' OR --Asmabec Spacehaler_Inha 100mcg (200 D) (0302000C0BIABBB)
bnf_code ='0302000C0BIACBC' OR --Asmabec Spacehaler_Inha 250mcg (200 D) (0302000C0BIACBC)
bnf_code ='0302000C0BIAABA' OR --Asmabec Spacehaler_Inha 50mcg (200 D) (0302000C0BIAABA)
bnf_code ='0302000R0BBAAAA' OR --Asmanex Twisthaler_D/P Inh 200mcg (30 D) (0302000R0BBAAAA)
bnf_code ='0302000R0BBABAB' OR --Asmanex Twisthaler_D/P Inh 200mcg (60 D) (0302000R0BBABAB)
bnf_code ='0302000R0BBACAC' OR --Asmanex Twisthaler_D/P Inh 400mcg (30 D) (0302000R0BBACAC)
bnf_code ='0302000R0BBADAD' OR --Asmanex Twisthaler_D/P Inh 400mcg (60 D) (0302000R0BBADAD)
bnf_code ='0302000C0BFAEAT' OR --Beclazone 100 E-Breathe_Inha 100mcg(200D (0302000C0BFAEAT)
bnf_code ='0302000C0BFABAB' OR --Beclazone 100_Inha 100mcg (200 D) (0302000C0BFABAB)
bnf_code ='0302000C0BFAGAR' OR --Beclazone 200_Inha 200mcg (200 D) (0302000C0BFAGAR)
bnf_code ='0302000C0BFAFAU' OR --Beclazone 250 E-Breathe_Inha 250mcg(200D (0302000C0BFAFAU)
bnf_code ='0302000C0BFACAC' OR --Beclazone 250_Inha 250mcg (200 D) (0302000C0BFACAC)
bnf_code ='0302000C0BFADAS' OR --Beclazone 50 E-Breathe_Inha 50mcg (200D) (0302000C0BFADAS)
bnf_code ='0302000C0BFAAAA' OR --Beclazone 50_Inha 50mcg (200 D) (0302000C0BFAAAA)
bnf_code ='0302000C0BBAIAU' OR --Becloforte E-Breathe_Inha 250mcg (200 D) (0302000C0BBAIAU)
bnf_code ='0302000C0BBAGAY' OR --Becloforte Integra_Inha 250mcg + Spacer (0302000C0BBAGAY)
bnf_code ='0302000C0BBAHAZ' OR --Becloforte Integra_Inha 250mcg Ref (0302000C0BBAHAZ)
bnf_code ='0302000C0BBABA0' OR --Becloforte-VM_Inha (200 D x 2)+Volumatic (0302000C0BBABA0)
bnf_code ='0302000C0BBAFA0' OR --Becloforte-VM_Inha (80 D) + Volumatic (0302000C0BBAFA0)
bnf_code ='0302000C0BBACAJ' OR --Becloforte_Disk 400mcg & Diskhaler (0302000C0BBACAJ)
bnf_code ='0302000C0BBADAQ' OR --Becloforte_Disk 400mcg Ref (0302000C0BBADAQ)
bnf_code ='0302000C0BBAJBT' OR --Becloforte_Inha 250mcg (180 D) (Import) (0302000C0BBAJBT)
bnf_code ='0302000C0BBAAAC' OR --Becloforte_Inha 250mcg (200 D) (0302000C0BBAAAC)
bnf_code ='0302000C0BBAEAX' OR --Becloforte_Inha 250mcg (80 D) (0302000C0BBAEAX)
bnf_code ='0302000C0AABXBX' OR --Beclomet Diprop/Formoterol_Inh100/6(120D (0302000C0AABXBX)
bnf_code ='0302000C0AAADAD' OR --Beclomet Diprop_Inh Susp 50mcg/ml (0302000C0AAADAD)
bnf_code ='0302000C0AABNBN' OR --Beclomet Diprop_Inha 100mcg (100 D) (0302000C0AABNBN)
bnf_code ='0302000C0AABFBF' OR --Beclomet Diprop_Inha 100mcg (200 D) CFF (0302000C0AABFBF)
bnf_code ='0302000C0AAABAB' OR --Beclomet Diprop_Inha 100mcg (200D) (0302000C0AAABAB)
bnf_code ='0302000C0AAAWAW' OR --Beclomet Diprop_Inha 100mcg (80 D) (0302000C0AAAWAW)
bnf_code ='0302000C0AABBBB' OR --Beclomet Diprop_Inha 100mcg 200 D+V/Spac (0302000C0AABBBB)
bnf_code ='0302000C0AABMBM' OR --Beclomet Diprop_Inha 200mcg (100 D) (0302000C0AABMBM)
bnf_code ='0302000C0AABVBV' OR --Beclomet Diprop_Inha 200mcg (200 D) CFF (0302000C0AABVBV)
bnf_code ='0302000C0AAARAR' OR --Beclomet Diprop_Inha 200mcg (200D) (0302000C0AAARAR)
bnf_code ='0302000C0AABTBT' OR --Beclomet Diprop_Inha 250mcg (180 D) (0302000C0AABTBT)
bnf_code ='0302000C0AABWBW' OR --Beclomet Diprop_Inha 250mcg (200 D) CFF (0302000C0AABWBW)
bnf_code ='0302000C0AAAZAZ' OR --Beclomet Diprop_Inha 250mcg (200 D) Ref (0302000C0AAAZAZ)
bnf_code ='0302000C0AAAYAY' OR --Beclomet Diprop_Inha 250mcg (200 D)+Spac (0302000C0AAAYAY)
bnf_code ='0302000C0AAACAC' OR --Beclomet Diprop_Inha 250mcg (200D) (0302000C0AAACAC)
bnf_code ='0302000C0AAAXAX' OR --Beclomet Diprop_Inha 250mcg (80 D) (0302000C0AAAXAX)
bnf_code ='0302000C0AABCBC' OR --Beclomet Diprop_Inha 250mcg 200 D+V/Spac (0302000C0AABCBC)
bnf_code ='0302000C0AABPBP' OR --Beclomet Diprop_Inha 400mcg (100 D) (0302000C0AABPBP)
bnf_code ='0302000C0AABEBE' OR --Beclomet Diprop_Inha 50mcg (200 D) CFF (0302000C0AABEBE)
bnf_code ='0302000C0AAAAAA' OR --Beclomet Diprop_Inha 50mcg (200D) (0302000C0AAAAAA)
bnf_code ='0302000C0AAAVAV' OR --Beclomet Diprop_Inha 50mcg (80 D) (0302000C0AAAVAV)
bnf_code ='0302000C0AABABA' OR --Beclomet Diprop_Inha 50mcg(200 D)+V/Spac (0302000C0AABABA)
bnf_code ='0302000C0AABQBQ' OR --Beclomet Diprop_Inha B/A 100mcg (100 D) (0302000C0AABQBQ)
bnf_code ='0302000C0AAATAT' OR --Beclomet Diprop_Inha B/A 100mcg (200 D) (0302000C0AAATAT)
bnf_code ='0302000C0AABRBR' OR --Beclomet Diprop_Inha B/A 200mcg (100 D) (0302000C0AABRBR)
bnf_code ='0302000C0AAAUAU' OR --Beclomet Diprop_Inha B/A 250mcg (200 D) (0302000C0AAAUAU)
bnf_code ='0302000C0AABSBS' OR --Beclomet Diprop_Inha B/A 400mcg (100 D) (0302000C0AABSBS)
bnf_code ='0302000C0AAASAS' OR --Beclomet Diprop_Inha B/A 50mcg (200 D) (0302000C0AAASAS)
bnf_code ='0302000C0AABGBG' OR --Beclomet Diprop_Inha B/A 50mcg(200 D)CFF (0302000C0AABGBG)
bnf_code ='0302000C0AABHBH' OR --Beclomet Diprop_Inha B/A100mcg(200 D)CFF (0302000C0AABHBH)
bnf_code ='0302000C0AABJBJ' OR --Beclomet Diprop_Pdr For Inh 100mcg(200 D (0302000C0AABJBJ)
bnf_code ='0302000C0AABUBU' OR --Beclomet Diprop_Pdr For Inh 200mcg(200D) (0302000C0AABUBU)
bnf_code ='0302000C0AABKBK' OR --Beclomet Diprop_Pdr For Inh 250mcg(100 D (0302000C0AABKBK)
bnf_code ='0302000C0AABIBI' OR --Beclomet Diprop_Pdr For Inh 50mcg(200 D) (0302000C0AABIBI)
bnf_code ='0302000C0AAAEAE' OR --Beclomet Diprop_Pdr For Inh Cap 100mcg (0302000C0AAAEAE)
bnf_code ='0302000C0AAAFAF' OR --Beclomet Diprop_Pdr For Inh Cap 200mcg (0302000C0AAAFAF)
bnf_code ='0302000C0AAAIAI' OR --Beclomet Diprop_Pdr For Inh Cap 400mcg (0302000C0AAAIAI)
bnf_code ='0302000C0AAALAL' OR --Beclomet Diprop_Pdr Inh 100mcg Disk Ref (0302000C0AAALAL)
bnf_code ='0302000C0AAAGAG' OR --Beclomet Diprop_Pdr Inh 100mcg Disk+Dev (0302000C0AAAGAG)
bnf_code ='0302000C0AAAKAK' OR --Beclomet Diprop_Pdr Inh 200mcg Disk Ref (0302000C0AAAKAK)
bnf_code ='0302000C0AAAHAH' OR --Beclomet Diprop_Pdr Inh 200mcg Disk+Dev (0302000C0AAAHAH)
bnf_code ='0302000C0AAAQAQ' OR --Beclomet Diprop_Pdr Inh 400mcg Disk Ref (0302000C0AAAQAQ)
bnf_code ='0302000C0AAAJAJ' OR --Beclomet Diprop_Pdr Inh 400mcg Disk+Dev (0302000C0AAAJAJ)
bnf_code ='0302000C0AABDBD' OR --Beclomet Diprop_Pdr Inh Cap 200mcg  @gn (0302000C0AABDBD)
bnf_code ='0302000C0AABYBY' OR --Beclomet/Formoterol_Inh 100/6 (120D) Dry (0302000C0AABYBY)
bnf_code ='0302000C0AABZBZ' OR --Beclomet/Formoterol_Inh 200/6 (120D) CFF (0302000C0AABZBZ)
bnf_code ='0302000C0AACACA' OR --Beclomet/Formoterol_Inh 200/6 (120D) Dry (0302000C0AACACA)
bnf_code ='0302000C0BMAAAE' OR --Beclometasone 100 Cyclocaps_Cap 100mcg (0302000C0BMAAAE)
bnf_code ='0302000C0BMABAF' OR --Beclometasone 200 Cyclocaps_Cap 200mcg (0302000C0BMABAF)
bnf_code ='0302000C0BMACAI' OR --Beclometasone 400 Cyclocaps_Cap 400mcg (0302000C0BMACAI)
bnf_code ='0302000C0BKAAAF' OR --Beclotaide_R/Cap 200mcg (0302000C0BKAAAF)
bnf_code ='0302000C0BHAAAG' OR --Becodisks_Disk 100mcg & Diskhaler (0302000C0BHAAAG)
bnf_code ='0302000C0BHABAL' OR --Becodisks_Disk 100mcg Ref (0302000C0BHABAL)
bnf_code ='0302000C0BHACAH' OR --Becodisks_Disk 200mcg & Diskhaler (0302000C0BHACAH)
bnf_code ='0302000C0BHADAK' OR --Becodisks_Disk 200mcg Ref (0302000C0BHADAK)
bnf_code ='0302000C0BHAEAJ' OR --Becodisks_Disk 400mcg & Diskhaler (0302000C0BHAEAJ)
bnf_code ='0302000C0BHAFAQ' OR --Becodisks_Disk 400mcg Ref (0302000C0BHAFAQ)
bnf_code ='0302000C0BCAUAT' OR --Becotide 100 E-Breathe_Inha 100mcg(200D) (0302000C0BCAUAT)
bnf_code ='0302000C0BCAHAB' OR --Becotide 100_Inha 100mcg (200 D) (0302000C0BCAHAB)
bnf_code ='0302000C0BCASAW' OR --Becotide 100_Inha 100mcg (80 D) (0302000C0BCASAW)
bnf_code ='0302000C0BCAQAR' OR --Becotide 200_Inha 200mcg (200 D) (0302000C0BCAQAR)
bnf_code ='0302000C0BCATAS' OR --Becotide 50 E-Breathe_Inha 50mcg (200 D) (0302000C0BCATAS)
bnf_code ='0302000C0BCAIAA' OR --Becotide 50_Inha 50mcg (200 D) (0302000C0BCAIAA)
bnf_code ='0302000C0BCARAV' OR --Becotide 50_Inha 50mcg (80 D) (0302000C0BCARAV)
bnf_code ='0302000C0BCAEAG' OR --Becotide_Becodisk 100mcg & Diskhaler (0302000C0BCAEAG)
bnf_code ='0302000C0BCAJAL' OR --Becotide_Becodisk 100mcg Ref (0302000C0BCAJAL)
bnf_code ='0302000C0BCAFAH' OR --Becotide_Becodisk 200mcg & Diskhaler (0302000C0BCAFAH)
bnf_code ='0302000C0BCAKAK' OR --Becotide_Becodisk 200mcg Ref (0302000C0BCAKAK)
bnf_code ='0302000C0BCANAJ' OR --Becotide_Becodisk 400mcg & Diskhaler (0302000C0BCANAJ)
bnf_code ='0302000C0BCAPAQ' OR --Becotide_Becodisk 400mcg Ref (0302000C0BCAPAQ)
bnf_code ='0302000C0BCACAE' OR --Becotide_R/Cap 100mcg (0302000C0BCACAE)
bnf_code ='0302000C0BCADAF' OR --Becotide_R/Cap 200mcg (0302000C0BCADAF)
bnf_code ='0302000C0BCAGAI' OR --Becotide_R/Cap 400mcg (0302000C0BCAGAI)
bnf_code ='0302000C0BCALAG' OR --Becotide_S/Pack 100mcg (0302000C0BCALAG)
bnf_code ='0302000C0BCABAD' OR --Becotide_Susp For Nebulisation 50mcg/ml (0302000C0BCABAD)
bnf_code ='0302000K0BFAAAV' OR --Budelin Novolizer_Inh 200mcg (100 D) (0302000K0BFAAAV)
bnf_code ='0302000K0BFABAC' OR --Budelin Novolizer_Inh 200mcg 100D Ref (0302000K0BFABAC)
bnf_code ='0302000K0BEAAAR' OR --Budesonide 200 Cyclocaps_Cap 200mcg (0302000K0BEAAAR)
bnf_code ='0302000K0BEABAS' OR --Budesonide 400 Cyclocaps_Cap 400mcg (0302000K0BEABAS)
bnf_code ='0302000K0AABBBB' OR --Budesonide/Formoterol_Inha200/6(120D)CFF (0302000K0AABBBB)
bnf_code ='0302000K0AAALAL' OR --Budesonide/Formoterol_InhaB/A 100/6(120D (0302000K0AAALAL)
bnf_code ='0302000K0AAAMAM' OR --Budesonide/Formoterol_InhaB/A 200/6(120D (0302000K0AAAMAM)
bnf_code ='0302000K0AABCBC' OR --Budesonide/Formoterol_InhaB/A 200/6(60D) (0302000K0AABCBC)
bnf_code ='0302000K0AAAUAU' OR --Budesonide/Formoterol_InhaB/A 400/12(60D (0302000K0AAAUAU)
bnf_code ='0302000K0AAAIAI' OR --Budesonide_Inh Soln 1mg/2ml Ud Vl (0302000K0AAAIAI)
bnf_code ='0302000K0AAAJAJ' OR --Budesonide_Inh Soln 500mcg/2ml Ud Vl (0302000K0AAAJAJ)
bnf_code ='0302000K0AAANAN' OR --Budesonide_Inh Soln 500mcg/ml 2ml Ud @gn (0302000K0AAANAN)
bnf_code ='0302000K0AAAQAQ' OR --Budesonide_Inh Susp 500mcg/ml 2ml Vl @gn (0302000K0AAAQAQ)
bnf_code ='0302000K0AAAZAZ' OR --Budesonide_Inha 100mcg (120 D) CFF (0302000K0AAAZAZ)
bnf_code ='0302000K0AAABAB' OR --Budesonide_Inha 200mcg (100 D) (0302000K0AAABAB)
bnf_code ='0302000K0AAACAC' OR --Budesonide_Inha 200mcg (100 D) Ref (0302000K0AAACAC)
bnf_code ='0302000K0AABABA' OR --Budesonide_Inha 200mcg (120 D) CFF (0302000K0AABABA)
bnf_code ='0302000K0AAADAD' OR --Budesonide_Inha 200mcg (200 D) (0302000K0AAADAD)
bnf_code ='0302000K0AAATAT' OR --Budesonide_Inha 200mcg (200 D) + Dev (0302000K0AAATAT)
bnf_code ='0302000K0AAAEAE' OR --Budesonide_Inha 200mcg (200 D) Ref (0302000K0AAAEAE)
bnf_code ='0302000K0AAAAAA' OR --Budesonide_Inha 50mcg (200 D) (0302000K0AAAAAA)
bnf_code ='0302000K0AAAFAF' OR --Budesonide_Inha 50mcg (200 D) Ref (0302000K0AAAFAF)
bnf_code ='0302000K0AAAKAK' OR --Budesonide_Pdr For Inh 100mcg (200 D) (0302000K0AAAKAK)
bnf_code ='0302000K0AAAGAG' OR --Budesonide_Pdr For Inh 200mcg (100 D) (0302000K0AAAGAG)
bnf_code ='0302000K0AAAWAW' OR --Budesonide_Pdr For Inh 200mcg (100D) Ref (0302000K0AAAWAW)
bnf_code ='0302000K0AAAVAV' OR --Budesonide_Pdr For Inh 200mcg (100D)+Dev (0302000K0AAAVAV)
bnf_code ='0302000K0AAAXAX' OR --Budesonide_Pdr For Inh 200mcg (200 D) (0302000K0AAAXAX)
bnf_code ='0302000K0AAAYAY' OR --Budesonide_Pdr For Inh 400mcg (100 D) (0302000K0AAAYAY)
bnf_code ='0302000K0AAAHAH' OR --Budesonide_Pdr For Inh 400mcg (50 D) (0302000K0AAAHAH)
bnf_code ='0302000K0AAAPAP' OR --Budesonide_Pdr For Inh 400mcg (50D)  @gn (0302000K0AAAPAP)
bnf_code ='0302000K0AAARAR' OR --Budesonide_Pdr For Inh Cap 200mcg (0302000K0AAARAR)
bnf_code ='0302000K0AAASAS' OR --Budesonide_Pdr For Inh Cap 400mcg (0302000K0AAASAS)
bnf_code ='0302000U0AAABAB' OR --Ciclesonide_Inh 160mcg (120 D) CFF (0302000U0AAABAB)
bnf_code ='0302000U0AAACAC' OR --Ciclesonide_Inh 160mcg (60 D) CFF (0302000U0AAACAC)
bnf_code ='0302000U0AAAAAA' OR --Ciclesonide_Inh 80mcg (120 D) CFF (0302000U0AAAAAA)
bnf_code ='0302000C0BPABBF' OR --Clenil Modulite_Inha 100mcg (200D) (0302000C0BPABBF)
bnf_code ='0302000C0BPACBV' OR --Clenil Modulite_Inha 200mcg (200D) (0302000C0BPACBV)
bnf_code ='0302000C0BPADBW' OR --Clenil Modulite_Inha 250mcg (200D) (0302000C0BPADBW)
bnf_code ='0302000C0BPAABE' OR --Clenil Modulite_Inha 50mcg (200D) (0302000C0BPAABE)
bnf_code ='0302000N0BLABBF' OR --Combisal_Inha 125mcg/25mcg (120 D) (0302000N0BLABBF)
bnf_code ='0302000N0BLACBG' OR --Combisal_Inha 250mcg/25mcg (120 D) (0302000N0BLACBG)
bnf_code ='0302000N0BLAABE' OR --Combisal_Inha 50mcg/25mcg (120 D) (0302000N0BLAABE)
bnf_code ='0302000K0BHAAAM' OR --DuoResp Spiromax_Inh 160mcg/4.5mcg(120D) (0302000K0BHAAAM)
bnf_code ='0302000K0BHABAU' OR --DuoResp Spiromax_Inh 320mcg/9mcg (60 D) (0302000K0BHABAU)
bnf_code ='0302000C0BNAABU' OR --Easyhaler_Beclomet 200mcg (200 D) (0302000C0BNAABU)
bnf_code ='0302000K0BGAAAK' OR --Easyhaler_Budesonide 100mcg (200 D) (0302000K0BGAAAK)
bnf_code ='0302000K0BGABAX' OR --Easyhaler_Budesonide 200mcg (200 D) (0302000K0BGABAX)
bnf_code ='0302000K0BGACAY' OR --Easyhaler_Budesonide 400mcg (100 D) (0302000K0BGACAY)
bnf_code ='0302000C0BGABAB' OR --Filair 100_Inha 100mcg (200 D) (0302000C0BGABAB)
bnf_code ='0302000C0BGAAAA' OR --Filair 50_Inha 50mcg (200 D) (0302000C0BGAAAA)
bnf_code ='0302000C0BGACAC' OR --Filair Fte_Inha 250mcg (200 D) (0302000C0BGACAC)
bnf_code ='0302000N0BBASAS' OR --Flixotide_Accuhaler 100mcg (60 D) (0302000N0BBASAS)
bnf_code ='0302000N0BBATAT' OR --Flixotide_Accuhaler 250mcg (60 D) (0302000N0BBATAT)
bnf_code ='0302000N0BBAUAU' OR --Flixotide_Accuhaler 500mcg (60 D) (0302000N0BBAUAU)
bnf_code ='0302000N0BBARAR' OR --Flixotide_Accuhaler 50mcg (60 D) (0302000N0BBARAR)
bnf_code ='0302000N0BBACAC' OR --Flixotide_Disk 100mcg & Diskhaler (0302000N0BBACAC)
bnf_code ='0302000N0BBADAD' OR --Flixotide_Disk 100mcg Ref (0302000N0BBADAD)
bnf_code ='0302000N0BBAEAE' OR --Flixotide_Disk 250mcg & Diskhaler (0302000N0BBAEAE)
bnf_code ='0302000N0BBAFAF' OR --Flixotide_Disk 250mcg Ref (0302000N0BBAFAF)
bnf_code ='0302000N0BBAPAP' OR --Flixotide_Disk 500mcg & Diskhaler (0302000N0BBAPAP)
bnf_code ='0302000N0BBAQAQ' OR --Flixotide_Disk 500mcg Ref (0302000N0BBAQAQ)
bnf_code ='0302000N0BBAAAA' OR --Flixotide_Disk 50mcg & Diskhaler (0302000N0BBAAAA)
bnf_code ='0302000N0BBABAB' OR --Flixotide_Disk 50mcg Ref (0302000N0BBABAB)
bnf_code ='0302000N0BBAXBA' OR --Flixotide_Evohaler 125mcg (120 D) (0302000N0BBAXBA)
bnf_code ='0302000N0BBAYBB' OR --Flixotide_Evohaler 125mcg (60 D) (0302000N0BBAYBB)
bnf_code ='0302000N0BBAZBC' OR --Flixotide_Evohaler 250mcg (120 D) (0302000N0BBAZBC)
bnf_code ='0302000N0BBBABD' OR --Flixotide_Evohaler 250mcg (60 D) (0302000N0BBBABD)
bnf_code ='0302000N0BBBBBH' OR --Flixotide_Evohaler 50mcg (120 D) (0302000N0BBBBBH)
bnf_code ='0302000N0BBAIAI' OR --Flixotide_Inha 125mcg (120 D) (0302000N0BBAIAI)
bnf_code ='0302000N0BBALAL' OR --Flixotide_Inha 125mcg (60 D) (0302000N0BBALAL)
bnf_code ='0302000N0BBAMAM' OR --Flixotide_Inha 250mcg (120 D) (0302000N0BBAMAM)
bnf_code ='0302000N0BBANAN' OR --Flixotide_Inha 250mcg (60 D) (0302000N0BBANAN)
bnf_code ='0302000N0BBAGAG' OR --Flixotide_Inha 25mcg (120 D) (0302000N0BBAGAG)
bnf_code ='0302000N0BBAJAJ' OR --Flixotide_Inha 25mcg (60 D) (0302000N0BBAJAJ)
bnf_code ='0302000N0BBAHAH' OR --Flixotide_Inha 50mcg (120 D) (0302000N0BBAHAH)
bnf_code ='0302000N0BBAKAK' OR --Flixotide_Inha 50mcg (60 D) (0302000N0BBAKAK)
bnf_code ='0302000N0AAAIAI' OR --Fluticasone Prop_Inha 125mcg (120 D) (0302000N0AAAIAI)
bnf_code ='0302000N0AABABA' OR --Fluticasone Prop_Inha 125mcg (120 D) CFF (0302000N0AABABA)
bnf_code ='0302000N0AAALAL' OR --Fluticasone Prop_Inha 125mcg (60 D) (0302000N0AAALAL)
bnf_code ='0302000N0AABBBB' OR --Fluticasone Prop_Inha 125mcg (60 D) CFF (0302000N0AABBBB)
bnf_code ='0302000N0AAAMAM' OR --Fluticasone Prop_Inha 250mcg (120 D) (0302000N0AAAMAM)
bnf_code ='0302000N0AABCBC' OR --Fluticasone Prop_Inha 250mcg (120 D) CFF (0302000N0AABCBC)
bnf_code ='0302000N0AAANAN' OR --Fluticasone Prop_Inha 250mcg (60 D) (0302000N0AAANAN)
bnf_code ='0302000N0AABDBD' OR --Fluticasone Prop_Inha 250mcg (60 D) CFF (0302000N0AABDBD)
bnf_code ='0302000N0AAAGAG' OR --Fluticasone Prop_Inha 25mcg (120 D) (0302000N0AAAGAG)
bnf_code ='0302000N0AAAJAJ' OR --Fluticasone Prop_Inha 25mcg (60 D) (0302000N0AAAJAJ)
bnf_code ='0302000N0AAAHAH' OR --Fluticasone Prop_Inha 50mcg (120 D) (0302000N0AAAHAH)
bnf_code ='0302000N0AABHBH' OR --Fluticasone Prop_Inha 50mcg (120 D) CFF (0302000N0AABHBH)
bnf_code ='0302000N0AAAKAK' OR --Fluticasone Prop_Inha 50mcg (60 D) (0302000N0AAAKAK)
bnf_code ='0302000N0AAASAS' OR --Fluticasone Prop_Pdr For Inh 100mcg(60D) (0302000N0AAASAS)
bnf_code ='0302000N0AAATAT' OR --Fluticasone Prop_Pdr For Inh 250mcg(60D) (0302000N0AAATAT)
bnf_code ='0302000N0AAAUAU' OR --Fluticasone Prop_Pdr For Inh 500mcg(60D) (0302000N0AAAUAU)
bnf_code ='0302000N0AAARAR' OR --Fluticasone Prop_Pdr For Inh 50mcg (60D) (0302000N0AAARAR)
bnf_code ='0302000N0AAADAD' OR --Fluticasone Prop_Pdr Inh 100mcg Disk Ref (0302000N0AAADAD)
bnf_code ='0302000N0AAACAC' OR --Fluticasone Prop_Pdr Inh 100mcg Disk+Dev (0302000N0AAACAC)
bnf_code ='0302000N0AABIBI' OR --Fluticasone Prop_Pdr Inh 250mcg Disk @gn (0302000N0AABIBI)
bnf_code ='0302000N0AAAFAF' OR --Fluticasone Prop_Pdr Inh 250mcg Disk Ref (0302000N0AAAFAF)
bnf_code ='0302000N0AAAEAE' OR --Fluticasone Prop_Pdr Inh 250mcg Disk+Dev (0302000N0AAAEAE)
bnf_code ='0302000N0AAAQAQ' OR --Fluticasone Prop_Pdr Inh 500mcg Disk Ref (0302000N0AAAQAQ)
bnf_code ='0302000N0AAAPAP' OR --Fluticasone Prop_Pdr Inh 500mcg Disk+Dev (0302000N0AAAPAP)
bnf_code ='0302000N0AAABAB' OR --Fluticasone Prop_Pdr Inh 50mcg Disk Ref (0302000N0AAABAB)
bnf_code ='0302000N0AAAAAA' OR --Fluticasone Prop_Pdr Inh 50mcg Disk+Dev (0302000N0AAAAAA)
bnf_code ='0302000N0AABJBJ' OR --Fluticasone/Formoterol_Inh 125/5mcg 120D (0302000N0AABJBJ)
bnf_code ='0302000N0AABKBK' OR --Fluticasone/Formoterol_Inh 250/10mcg120D (0302000N0AABKBK)
bnf_code ='0302000N0AABLBL' OR --Fluticasone/Formoterol_Inh 50/5mcg 120 D (0302000N0AABLBL)
bnf_code ='0302000N0AABQBQ' OR --Fluticasone/Formoterol_Inh BA50/5mcg120D (0302000N0AABQBQ)
bnf_code ='0302000N0AABPBP' OR --Fluticasone/Formoterol_InhBA125/5mcg120D (0302000N0AABPBP)
bnf_code ='0302000N0AAAXAX' OR --Fluticasone/Salmeterol_Inh 100/50mcg 60D (0302000N0AAAXAX)
bnf_code ='0302000N0AABFBF' OR --Fluticasone/Salmeterol_Inh 125/25mcg120D (0302000N0AABFBF)
bnf_code ='0302000N0AABGBG' OR --Fluticasone/Salmeterol_Inh 250/25mcg120D (0302000N0AABGBG)
bnf_code ='0302000N0AAAYAY' OR --Fluticasone/Salmeterol_Inh 250/50mcg 60D (0302000N0AAAYAY)
bnf_code ='0302000N0AABEBE' OR --Fluticasone/Salmeterol_Inh 50/25mcg 120D (0302000N0AABEBE)
bnf_code ='0302000N0AAAZAZ' OR --Fluticasone/Salmeterol_Inh 500/50mcg 60D (0302000N0AAAZAZ)
bnf_code ='0302000V0AAAAAA' OR --Fluticasone/Vilanterol_Inha 184/22mcg30D (0302000V0AAAAAA)
bnf_code ='0302000V0AAABAB' OR --Fluticasone/Vilanterol_Inha 92/22mcg 30D (0302000V0AAABAB)
bnf_code ='0302000N0BDAABJ' OR --Flutiform_Inha 125/5mcg (120 D) (0302000N0BDAABJ)
bnf_code ='0302000N0BDABBK' OR --Flutiform_Inha 250/10mcg (120 D) (0302000N0BDABBK)
bnf_code ='0302000N0BDACBL' OR --Flutiform_Inha 50/5mcg (120 D) (0302000N0BDACBL)
bnf_code ='0302000N0BDADBP' OR --Flutiform_K-haler 125/5mcg (120 D) (0302000N0BDADBP)
bnf_code ='0302000N0BDAEBQ' OR --Flutiform_K-haler 50/5mcg (120 D) (0302000N0BDAEBQ)
bnf_code ='0302000K0BIACAM' OR --Fobumix_Easyhaler 160mcg/4.5mcg (120 D) (0302000K0BIACAM)
bnf_code ='0302000K0BIABBC' OR --Fobumix_Easyhaler 160mcg/4.5mcg (60 D) (0302000K0BIABBC)
bnf_code ='0302000K0BIAAAU' OR --Fobumix_Easyhaler 320mcg/9mcg (60 D) (0302000K0BIAAAU)
bnf_code ='0302000K0BIADAL' OR --Fobumix_Easyhaler 80mcg/4.5mcg (120 D) (0302000K0BIADAL)
bnf_code ='0302000C0BRAABY' OR --Fostair NEXThaler_Inh 100mcg/6mcg (120D) (0302000C0BRAABY)
bnf_code ='0302000C0BRABCA' OR --Fostair NEXThaler_Inh 200mcg/6mcg (120D) (0302000C0BRABCA)
bnf_code ='0302000C0BQAABX' OR --Fostair_Inh 100mcg/6mcg (120D) CFF (0302000C0BQAABX)
bnf_code ='0302000C0BQABBZ' OR --Fostair_Inh 200mcg/6mcg (120D) CFF (0302000C0BQABBZ)
bnf_code ='0302000N0BKAAAY' OR --Fusacomb Easyhaler_Inha 250/50mcg (60 D) (0302000N0BKAAAY)
bnf_code ='0302000N0BKABAZ' OR --Fusacomb Easyhaler_Inha 500/50mcg (60 D) (0302000N0BKABAZ)
bnf_code ='0302000C0BSABBF' OR --Kelhale_Inha 100mcg (200 D) (0302000C0BSABBF)
bnf_code ='0302000C0BSAABE' OR --Kelhale_Inha 50mcg (200 D) (0302000C0BSAABE)
bnf_code ='0302000R0AAAAAA' OR --Mometasone Fur_Pdr For Inh 200mcg (30 D) (0302000R0AAAAAA)
bnf_code ='0302000R0AAABAB' OR --Mometasone Fur_Pdr For Inh 200mcg (60 D) (0302000R0AAABAB)
bnf_code ='0302000R0AAACAC' OR --Mometasone Fur_Pdr For Inh 400mcg (30 D) (0302000R0AAACAC)
bnf_code ='0302000R0AAADAD' OR --Mometasone Fur_Pdr For Inh 400mcg (60 D) (0302000R0AAADAD)
bnf_code ='0302000K0BCAAAH' OR --Pulmaxan_Turbohaler 400mcg (50 D) (0302000K0BCAAAH)
bnf_code ='0302000K0BBAEAA' OR --Pulmicort L.S._Inha 50mcg (200 D) (0302000K0BBAEAA)
bnf_code ='0302000K0BBAFAF' OR --Pulmicort L.S._Inha 50mcg (200 D) Ref (0302000K0BBAFAF)
bnf_code ='0302000K0BBAMAZ' OR --Pulmicort_Inha 100mcg (120 D) CFF (0302000K0BBAMAZ)
bnf_code ='0302000K0BBAAAB' OR --Pulmicort_Inha 200mcg (100 D) (0302000K0BBAAAB)
bnf_code ='0302000K0BBABAC' OR --Pulmicort_Inha 200mcg (100 D) Ref (0302000K0BBABAC)
bnf_code ='0302000K0BBANBA' OR --Pulmicort_Inha 200mcg (120 D) CFF (0302000K0BBANBA)
bnf_code ='0302000K0BBADAD' OR --Pulmicort_Inha 200mcg (200 D) (0302000K0BBADAD)
bnf_code ='0302000K0BBACAE' OR --Pulmicort_Inha 200mcg (200 D) Ref (0302000K0BBACAE)
bnf_code ='0302000K0BBALAT' OR --Pulmicort_Inha 200mcg (200D)+NebuChamber (0302000K0BBALAT)
bnf_code ='0302000K0BBAJAJ' OR --Pulmicort_Respule 250mcg/ml 2ml Ud (0302000K0BBAJAJ)
bnf_code ='0302000K0BBAGAI' OR --Pulmicort_Respule 500mcg/ml 2ml Ud (0302000K0BBAGAI)
bnf_code ='0302000K0BBAKAK' OR --Pulmicort_Turbohaler 100mcg (200 D) (0302000K0BBAKAK)
bnf_code ='0302000K0BBAHAG' OR --Pulmicort_Turbohaler 200mcg (100 D) (0302000K0BBAHAG)
bnf_code ='0302000K0BBAIAH' OR --Pulmicort_Turbohaler 400mcg (50 D) (0302000K0BBAIAH)
bnf_code ='0302000C0BLABBN' OR --Pulvinal Beclomet_Inha 100mcg (100 D) (0302000C0BLABBN)
bnf_code ='0302000C0BLAABM' OR --Pulvinal Beclomet_Inha 200mcg (100 D) (0302000C0BLAABM)
bnf_code ='0302000C0BLACBP' OR --Pulvinal Beclomet_Inha 400mcg (100 D) (0302000C0BLACBP)
bnf_code ='0302000C0BJAFBH' OR --Qvar 100 E-Breathe_Inha 100mcg (200 D) (0302000C0BJAFBH)
bnf_code ='0302000C0BJADBH' OR --Qvar 100_Autohaler 100mcg (200 D) (0302000C0BJADBH)
bnf_code ='0302000C0BJABBF' OR --Qvar 100_Inha 100mcg (200 D) (0302000C0BJABBF)
bnf_code ='0302000C0BJAEBG' OR --Qvar 50 E-Breathe_Inha 50mcg (200 D) (0302000C0BJAEBG)
bnf_code ='0302000C0BJACBG' OR --Qvar 50_Autohaler 50mcg (200 D) (0302000C0BJACBG)
bnf_code ='0302000C0BJAABE' OR --Qvar 50_Inha 50mcg (200 D) (0302000C0BJAABE)
bnf_code ='0302000V0BBAAAA' OR --Relvar Ellipta_Inha 184mcg/22mcg (30 D) (0302000V0BBAAAA)
bnf_code ='0302000V0BBABAB' OR --Relvar Ellipta_Inha 92mcg/22mcg (30 D) (0302000V0BBABAB)
bnf_code ='0302000N0BIAABF' OR --Sereflo_Inh 125mcg/25mcg (120D) (0302000N0BIAABF)
bnf_code ='0302000N0BIABBG' OR --Sereflo_Inh 250mcg/25mcg (120D) (0302000N0BIABBG)
bnf_code ='0302000N0BCAAAX' OR --Seretide 100_Accuhaler 100mcg/50mcg(60D) (0302000N0BCAAAX)
bnf_code ='0302000N0BCAEBF' OR --Seretide 125_Evohaler 125mcg/25mcg(120D) (0302000N0BCAEBF)
bnf_code ='0302000N0BCABAY' OR --Seretide 250_Accuhaler 250mcg/50mcg(60D) (0302000N0BCABAY)
bnf_code ='0302000N0BCAFBG' OR --Seretide 250_Evohaler 250mcg/25mcg(120D) (0302000N0BCAFBG)
bnf_code ='0302000N0BCACAZ' OR --Seretide 500_Accuhaler 500mcg/50mcg(60D) (0302000N0BCACAZ)
bnf_code ='0302000N0BCADBE' OR --Seretide 50_Evohaler 50mcg/25mcg (120 D) (0302000N0BCADBE)
bnf_code ='0302000N0BFAABF' OR --Sirdupla_Inh 125mcg/25mcg (120D) (0302000N0BFAABF)
bnf_code ='0302000N0BFABBG' OR --Sirdupla_Inh 250mcg/25mcg (120D) (0302000N0BFABBG)
bnf_code ='0302000K0BDADBB' OR --Symbicort_Inh Pressurised 200/6mcg(120D) (0302000K0BDADBB)
bnf_code ='0302000K0BDAAAL' OR --Symbicort_Turbohaler 100mcg/6mcg (120 D) (0302000K0BDAAAL)
bnf_code ='0302000K0BDABAM' OR --Symbicort_Turbohaler 200mcg/6mcg (120 D) (0302000K0BDABAM)
bnf_code ='0302000K0BDACAU' OR --Symbicort_Turbohaler 400mcg/12mcg (60 D) (0302000K0BDACAU)
bnf_code ='0302000V0BCAAA0' OR --Trelegy Ellipta_Inha 92/55/22mcg (30 D) (0302000V0BCAAA0)
bnf_code ='0301011ABBBAAA0' OR --Trimbow_Inh 87mcg/5mcg/9mcg (120 D) (0301011ABBBAAA0)
bnf_code ='0302000C0BDAAAN' OR --Ventide_Inha (200 D) (0302000C0BDAAAN)
bnf_code ='0302000C0BDABAP' OR --Ventide_R/Cap 400mcg (0302000C0BDABAP)
bnf_code ='0302000C0BDACBL' --Ventide_R/Cap Paed 200mcg (0302000C0BDACBL)
    GROUP BY
      bnf_name,
      bnf_code
    ORDER BY
      items DESC
"""
bsa_ics_all_df = bq.cached_read(sql, csv_path='bsa_ics_all_df.csv')
bsa_ics_all_df2 = bsa_ics_all_df

In [152]:
#merge high dose dataframes to show differences
ics_high_dose_df = dl_ics_high_dose_df.merge(bsa_ics_high_dose_df, on="bnf_code", how="outer", indicator=True) #merge both df
dl_ics_high_dose_only_df = ics_high_dose_df[ics_high_dose_df['_merge'].isin(['left_only'])]  #create dl only df (left hand side of merge)
dl_ics_high_dose_only_df = dl_ics_high_dose_only_df.drop(columns=['bnf_name_y', 'items_y', '_merge']) #drop unwanted columns
bsa_ics_high_dose_only_df = ics_high_dose_df[ics_high_dose_df['_merge'].isin(['right_only'])]  #create bsa only df (right hand side of merge)
bsa_ics_high_dose_only_df = bsa_ics_high_dose_only_df.drop(columns=['bnf_name_x', 'items_x', '_merge']) #drop unwanted columns

### High Dose ICS which are only in one dataset

**High Dose ICS which are in DataLab measure but not ePACT2**

In [153]:
display(dl_ics_high_dose_only_df)

,bnf_name_x,bnf_code,items_x
13,AirFluSal Forspiro_Inh 500/50mcg (60D),0302000N0BGAAAZ,5697.0
17,Beclomet/Formoterol_Inh 200/6 (120D) CFF,0302000C0AABZBZ,2407.0
18,Combisal_Inha 250mcg/25mcg (120 D),0302000N0BLACBG,2303.0
19,Fluticasone/Vilanterol_Inha 184/22mcg30D,0302000V0AAAAAA,2251.0
22,Ciclesonide_Inh 160mcg (60 D) CFF,0302000U0AAACAC,1492.0
23,Budesonide_Pdr For Inh 400mcg (50 D),0302000K0AAAHAH,1285.0
24,Ciclesonide_Inh 160mcg (120 D) CFF,0302000U0AAABAB,1092.0
28,Alvesco 160_Inh 160mcg (60 D) CFF,0302000U0BBACAC,390.0
30,Alvesco 160_Inh 160mcg (120 D) CFF,0302000U0BBABAB,278.0
32,Fusacomb Easyhaler_Inha 500/50mcg (60 D),0302000N0BKABAZ,181.0


**High Dose ICS which are in ePACT2 measure but not DataLab**

In [121]:
display(bsa_ics_high_dose_only_df)

,bnf_code,bnf_name_y,items_y
36,0302000C0BQAABX,Fostair_Inh 100mcg/6mcg (120D) CFF,318552.0
37,0302000C0BRAABY,Fostair NEXThaler_Inh 100mcg/6mcg (120D),42538.0
38,0302000N0BCABAY,Seretide 250_Accuhaler 250mcg/50mcg(60D),22254.0
39,0302000C0BRABCA,Fostair NEXThaler_Inh 200mcg/6mcg (120D),15338.0
40,0302000N0AAAYAY,Fluticasone/Salmeterol_Inh 250/50mcg 60D,5457.0
41,0302000N0AAATAT,Fluticasone Prop_Pdr For Inh 250mcg(60D),816.0
42,0302000N0BBATAT,Flixotide_Accuhaler 250mcg (60 D),807.0
43,0302000K0BGACAY,Easyhaler_Budesonide 400mcg (100 D),411.0
44,0302000R0AAADAD,Mometasone Fur_Pdr For Inh 400mcg (60 D),64.0
45,0302000R0AAACAC,Mometasone Fur_Pdr For Inh 400mcg (30 D),39.0


### All ICS which are not in both datasets

In [122]:
#merge high dose dataframes to show differences
ics_all_df = dl_ics_all_df.merge(bsa_ics_all_df, on="bnf_code", how="outer", indicator=True) #merge both df
dl_ics_all_df = ics_all_df[ics_all_df['_merge'].isin(['left_only'])]  #create dl only df (left hand side of merge)
dl_ics_all_df = dl_ics_all_df.drop(columns=['bnf_name_y', 'items_y', '_merge']) #drop unwanted columns
bsa_ics_all_df = ics_all_df[ics_all_df['_merge'].isin(['right_only'])]  #create bsa only df (right hand side of merge)
bsa_ics_all_df = bsa_ics_all_df.drop(columns=['bnf_name_x', 'items_x', '_merge']) #drop unwanted columns

**ICS which are in DataLab measure but not ePACT2**

In [123]:
display(dl_ics_all_df)

,bnf_name_x,bnf_code,items_x
86,Soprobec_Inha 100mcg (200 D),0302000C0BTACBF,1043.0
104,Soprobec_Inha 50mcg (200 D),0302000C0BTAABE,276.0
112,Soprobec_Inha 200mcg (200 D),0302000C0BTABBV,150.0
118,Fluticasone Prop_Inh Soln 500mcg/2ml Ud,0302000N0AAAVAV,121.0
122,Fluticasone Prop_Inh Soln 2mg/2ml Ud,0302000N0AAAWAW,81.0
128,Flixotide_Nebules 500mcg/2ml Ud,0302000N0BBAVAV,52.0
132,Stalpex_Inha 500/50mcg (60 D),0302000N0BMAAAZ,33.0
134,Flixotide_Nebules 2mg/2ml Ud,0302000N0BBAWAW,28.0
137,Soprobec_Inha 250mcg (200 D),0302000C0BTADBW,10.0


**ICS which are in ePACT2 measure but not DataLab**

In [125]:
display(bsa_ics_all_df)

,bnf_code,bnf_name_y,items_y
145,0301011ABBBAAA0,Trimbow_Inh 87mcg/5mcg/9mcg (120 D),67096.0


### Difference in code on calculations

**Percentage high dose in DataLab measure**

In [147]:
100* dl_ics_high_dose_df['items'].sum()/dl_ics_all_df2['items'].sum()

20.057355019564977

**Percentage high dose in ePACT2 measure**

In [148]:
100*bsa_ics_high_dose_df['items'].sum()/bsa_ics_all_df2['items'].sum()

40.219826559451995

**Discussion**

There appears to be significant variation in the codes between ePACT2 and the DataLab, with double the number of high-dose ICS items appearing in the ePACT2 version.

Having compared the codes unique to each of the measures, most of the codes which are in the ePACT2 measure but not the DataLab version relate to inhalers which are *not* considered high-dose by BTS/SIGN.

These are:

| bnf_code | bnf_name | items |
| --- | --- | --- |
0302000C0BQAABX | Fostair_Inh 100mcg/6mcg (120D) CFF | 318552
| 0302000C0BRAABY | Fostair NEXThaler_Inh 100mcg/6mcg (120D) | 42538
| 0302000N0BCABAY | Seretide 250_Accuhaler 250mcg/50mcg(60D) | 22254
| 0302000N0AAAYAY | Fluticasone/Salmeterol_Inh 250/50mcg 60D | 5457
| 0302000N0AAATAT | Fluticasone Prop_Pdr For Inh 250mcg(60D) | 816
| 0302000N0BBATAT | Flixotide_Accuhaler 250mcg (60 D) | 807
| 0302000R0AAADAD | Mometasone Fur_Pdr For Inh 400mcg (60 D) | 64
| 0302000R0AAACAC | Mometasone Fur_Pdr For Inh 400mcg (30 D) | 39
| 0302000R0BBADAD | Asmanex Twisthaler_D/P Inh 400mcg (60 D) | 28
| 0302000R0BBACAC | Asmanex Twisthaler_D/P Inh 400mcg (30 D) | 26

The other two items (0302000C0BRABCA Fostair NEXThaler_Inh 200mcg/6mcg (120D) and 0302000K0BGACAY Easyhaler_Budesonide 400mcg (100 D)) appear to be correct, and should be added to the DataLab high dose numerator, as the generic/brand wildcard code.

We would suggest that NHSBSA also review their measure, and consider removing the items above, and adding in other preparations as identified in *High Dose ICS which are in DataLab measure but not ePACT2*

As part of this analysis, we have also found some products in the DataLab measure denominator which should be removed.  These are:

| bnf_name| bnf_code | items |
| --- | --- | --- |
| 0302000N0AAAVAV | Fluticasone Prop_Inh Soln 500mcg/2ml Ud | 121
| 0302000N0AAAWAW | Fluticasone Prop_Inh Soln 2mg/2ml Ud | 81
| 0302000N0BBAVAV | Flixotide_Nebules 500mcg/2ml Ud |  52
| 0302000N0BBAWAW | Flixotide_Nebules 2mg/2ml Ud | 28

These are nebules, and therefore not usual inhaled corticosteroid treatment for most patients.

For the denominator, we found that the DataLab codes are missing: 0301011ABBBAAA0 Trimbow_Inh 87mcg/5mcg/9mcg (120 D), and therefore we will add this into the denominator (at chemical substance level).

We will therefore make the following changes to the DataLab measure code for high-dose ICS:

**Numerator**

*Additions:*

`0302000C0%CA --Beclomet/Formoterol_Inh 200/6 (120D) Dry (brands and generic)`

`0302000K0%AY --Budesonide_Pdr For Inh 400mcg (100 D) (brands and generic)`


**Denominator**

*Additions:*

`0301011AB% --BeclometDiprop/Formoterol/Glycopyrronium`

*Deletions:*

`0302000N0%AV --Fluticasone Prop_Inh Soln 500mcg/2ml Ud (brands and generic)`

`0302000N0%AW --Fluticasone Prop_Inh Soln 2mg/2ml Ud (brands and generic)`

### Proposed new measure code

``
{
  "name": "High dose inhaled corticosteroids",
  "title": [
    "high dose inhaled corticosteroids",
    "compared with prescribing of all inhaled corticosteroids"
  ],
  "description": [
    "Prescribing of high dose inhaled corticosteroids",
    "compared with prescribing of all inhaled corticosteroids"
  ],
  "why_it_matters": [
    "<a href='https://www.brit-thoracic.org.uk/standards-of-care/guidelines/btssign-british-guideline-on-the-management-of-asthma/'> Latest BTS/SIGN guidance </a> on the treatment of asthma recommends that ",
    "patients should be maintained at the lowest possible dose of inhaled corticosteroid.",
    "Reduction in inhaled corticosteroid dose should be slow as patients deteriorate at different rates. ",
    "Reductions should be considered every three months, decreasing the dose by approximately 25–50% each time. ",
    "'High-dose' inhaled corticosteroids are listed at step 4 of the guidelines. ",
    "<a href='https://goldcopd.org/wp-content/uploads/2016/12/wms-GOLD-2017-Pocket-Guide.pdf'>The latest guidance for treatment ",
    "of COPD</a> now recommends use of another treatment in preference to inhaled corticosteroids. ",
    "There is some evidence that inhaled corticosteroids increases the risk of pneumonia.  This risk appears to increase with dose."
  ],
  "numerator_short": "High dose ICS items",
  "denominator_short": "All ICS items",
  "url": null,
  "is_percentage": true,
  "is_cost_based": false,
  "low_is_good": true,
  "tags": [
    "core",
    "respiratory",
    "safety"
  ],
  "numerator_type": "bnf_items",
  "numerator_where": [
    "(",
    "bnf_code LIKE '0302000C0%AC' OR --Beclomet Diprop_Inha 250mcg (200D) (brands and generic) \n",
    "bnf_code LIKE '0302000C0%AU' OR --Beclomet Diprop_Inha B/A 250mcg (200 D) (brands and generic) \n",
    "bnf_code LIKE '0302000C0%BK' OR --Beclomet Diprop_Pdr For Inh 250mcg(100 D (brands and generic) \n",
    "bnf_code LIKE '0302000C0%BW' OR --Beclomet Diprop_Inha 250mcg (200 D) CFF (brands and generic) \n",
    "bnf_code LIKE '0302000C0%BZ' OR --Beclomet/Formoterol_Inh 200/6 (120D) CFF (brands and generic) \n",
    "bnf_code LIKE '0302000C0%CA' OR --Beclomet/Formoterol_Inh 200/6 (120D) Dry (brands and generic) \n",
    "bnf_code LIKE '0302000K0%AH' OR --Budesonide_Pdr For Inh 400mcg (50 D) (brands and generic) \n",
    "bnf_code LIKE '0302000K0%AY' OR --Budesonide_Pdr For Inh 400mcg (100 D) (brands and generic) \n",
    "bnf_code LIKE '0302000K0%AU' OR --Budesonide/Formoterol_InhaB/A 400/12(60D (brands and generic) \n",
    "bnf_code LIKE '0302000N0%AF' OR --Fluticasone Prop_Pdr Inh 250mcg Disk Ref (brands and generic) \n",
    "bnf_code LIKE '0302000N0%AP' OR --Fluticasone Prop_Pdr Inh 500mcg Disk+Dev (brands and generic) \n",
    "bnf_code LIKE '0302000N0%AU' OR --Fluticasone Prop_Pdr For Inh 500mcg(60D) (brands and generic) \n",
    "bnf_code LIKE '0302000N0%AZ' OR --Fluticasone/Salmeterol_Inh 500/50mcg 60D (brands and generic) \n",
    "bnf_code LIKE '0302000N0%BC' OR --Fluticasone Prop_Inha 250mcg (120 D) CFF (brands and generic) \n",
    "bnf_code LIKE '0302000N0%BG' OR --Fluticasone/Salmeterol_Inh 250/25mcg120D (brands and generic) \n",
    "bnf_code LIKE '0302000N0%BK' OR --Fluticasone/Formoterol_Inh 250/10mcg120D (brands and generic) \n",
    "bnf_code LIKE '0302000U0%AB' OR --Ciclesonide_Inh 160mcg (120 D) CFF (brands and generic) \n",
    "bnf_code LIKE '0302000U0%AC' OR --Ciclesonide_Inh 160mcg (60 D) CFF (brands and generic) \n",
    "bnf_code LIKE '0302000V0%AA'    --Fluticasone/Vilanterol_Inha 184/22mcg30D (brands and generic) \n",
    ")"
  ],
  "denominator_type": "bnf_items",
  "denominator_where": [
    "substr(bnf_code,0,9) IN (",
      "'0302000C0', --Beclometasone Dipropionate \n",
      "'0302000K0', --Budesonide \n",
      "'0302000N0', --Fluticasone Propionate (Inh) \n",
      "'0302000R0', --Mometasone Furoate \n",
      "'0302000U0', --Ciclesonide \n",
      "'0302000V0', --Fluticasone Furoate (Inh) \n",
      "'0301011AB') --BeclometDiprop/Formoterol/Glycopyrronium \n",
    "AND ",
    "bnf_code NOT LIKE '0302000N0%AV' AND --Fluticasone Prop_Inh Soln 500mcg/2ml Ud (brands and generic) \n",
    "bnf_code NOT LIKE '0302000N0%AW' --Fluticasone Prop_Inh Soln 2mg/2ml Ud (brands and generic)"
  ]
}
``